# Chronic Absenteeism Rate Prediction (CARP) Extract Transform and Load (ETL)

### Import Python libraries for data manipulation

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import re

### Import IBM Watson Studio utility library, initialize product credentials and define function for downloading project files

In [2]:
# The code was removed by Watson Studio for sharing.

In [3]:
# function to retrieve project assets 
def download(project_file_name,project=None):    
    # get the file
    print("Attempting to get file {}".format(project_file_name))
    _bytes = project.get_file(project_file_name).read()

    print("Downloading...")
    
    with open(project_file_name, 'wb') as f: 
        f.write(bytearray(_bytes))
        print("Completed writing out file")
        
    return 0

### Define functions and patterns for data quality assessment and cleansing

In [4]:
def valid_number(x):
    if x in [np.nan, np.inf, np.NINF]:
        return False
    if type(x) in [np.int, np.float]:
        return True
    try:
        float(x)
        return True
    except ValueError:
        return False

def assess(df):
    return int(np.around(df.applymap(valid_number).all(1).to_frame().rename(columns={0:'valid'}).query('valid').shape[0]/df.shape[0] * 100))

not_num = re.compile('[^0-9\.]')
one_not_num = re.compile('^[^0-9\.]$')

def clean(df):
    return df.replace(one_not_num,'0').replace(not_num,'').astype('float')
valid_number(np.inf)


def valid_pct(valid_list, df_name):
    Pct_Valid = assess(eval(df_name))
    valid_list.append(dict(DataFrame=df_name, Pct_Valid=Pct_Valid))
    return Pct_Valid

valid_list = []

### Process chronic absenteeism rate data from California Department of Education

In [5]:
abs_file_name = 'la_county_chronic_absenteeism_2017-2018.csv'
download(abs_file_name,project)
abs_df = pd.read_csv(abs_file_name,usecols=[3,4,5,7,13],names=['Year','Percent', 'Count', 'Tract','Total'])
abs_18 = abs_df.query("Year == '2018'").copy()

abs_18.loc[:,'Tract']=abs_18.Tract.astype(str)
abs_18['Tract_Nbr'] = abs_18.Tract.astype(int)
abs_18.set_index('Tract',inplace=True)
print (f"Valid Rows Before Cleansing: {valid_pct(valid_list,'abs_18')}%")

abs_18.dropna(inplace=True)
abs_18.Percent=abs_18.Percent.astype(float)
abs_18.Total=abs_18.Total.str.replace(one_not_num,'0').str.replace(not_num,'').astype('int64')
abs_18.Count=abs_18.Count.str.replace(one_not_num,'0').str.replace(not_num,'').astype('int64')
print('\nabs_18 (initial):\n',abs_18.head())
abs_18.shape

Attempting to get file la_county_chronic_absenteeism_2017-2018.csv
Downloading...
Completed writing out file
Valid Rows Before Cleansing: 84%

abs_18 (initial):
         Year    Percent  Count  Total  Tract_Nbr
Tract                                           
221402  2018  10.526316     26    247     221402
197300  2018   9.053498     22    243     197300
134521  2018  10.631229     32    301     134521
571300  2018  10.779817     94    872     571300
572100  2018   7.142857     12    168     572100


(2158, 5)

### Prepare American Community Survey (ACS) 2013-2017 median income in last 12 months data

In [6]:
inc_file_name = 'median_income_2017.csv'
download(inc_file_name,project)
inc_17 = pd.read_csv(inc_file_name,skiprows=2,header=None,usecols=[1,3],names=['Tract','Income'])
inc_17.loc[:,'Tract']=inc_17.Tract.astype(str).str.slice(start=4)
inc_17.set_index('Tract',inplace=True)
print (f"Valid Rows Before Cleansing: {valid_pct(valid_list,'inc_17')}%")
inc_17.dropna(inplace=True)
inc_17.drop_duplicates(inplace=True)
inc_17.Income=inc_17.Income.str.replace(one_not_num,'0').str.replace(not_num,'').astype('int64')
print('\ninc_17:\n',inc_17.head())
inc_17.shape

Attempting to get file median_income_2017.csv
Downloading...
Completed writing out file
Valid Rows Before Cleansing: 99%

inc_17:
         Income
Tract         
400100  208393
400200  147500
400300   88173
400400  102821
400500   92375


(7174, 1)

### Prepare ACS 2013-2017 educational attainment data

In [7]:
edu_file_name = 'educ_attainment_by_tract_2017.csv'
download(edu_file_name,project)
edu_17=pd.read_csv(edu_file_name,usecols=['GEO.id2','HC02_EST_VC17','HC02_EST_VC18'])
edu_17.columns=['Tract', 'Pct_HS', 'Pct_Bach']
edu_17.drop([0,1],inplace=True)
edu_17.loc[:,'Tract']=edu_17.Tract.astype(str).str.slice(start=5)
edu_17.set_index('Tract',inplace=True)
print (f"Valid Rows Before Cleansing: {valid_pct(valid_list,'edu_17')}%")
edu_17=clean(edu_17)
edu_17.drop_duplicates(inplace=True)
print('\nedu_17:\n',edu_17.head())
edu_17.shape

Attempting to get file educ_attainment_by_tract_2017.csv
Downloading...
Completed writing out file
Valid Rows Before Cleansing: 99%

edu_17:
         Pct_HS  Pct_Bach
Tract                   
101110    78.6      21.5
101122    91.8      25.7
101210    74.1      16.6
101220    79.4      18.7
101300    86.1      30.3


(2306, 2)

### Prepare ACS 2013-2017 English language mastery data 

In [8]:
eng_file_name = 'english_mastery_by_tract_2017.csv'
download(eng_file_name,project)
eng_17=pd.read_csv(eng_file_name,usecols=[1,9],names=['Tract','Pct_Eng'])
eng_17.drop([0,1],inplace=True)
eng_17.loc[:,'Tract']=eng_17.Tract.astype(str).str.slice(start=4)
eng_17.set_index('Tract',inplace=True)
print (f"Valid Rows Before Cleansing: {valid_pct(valid_list,'eng_17')}%")
eng_17=eng_17.query("Pct_Eng !='-'")
eng_17.drop_duplicates(inplace=True)
eng_17=clean(eng_17)
print('\neng_17:\n',eng_17.head())
eng_17.shape

Attempting to get file english_mastery_by_tract_2017.csv
Downloading...
Completed writing out file
Valid Rows Before Cleansing: 99%

eng_17:
         Pct_Eng
Tract          
101110     78.3
101122     88.7
101210     64.0
101220     68.2
101300     85.6


(563, 1)

### Prepare ACS 2013-2017 race of householder data 

In [9]:
race_file_name = 'race_of_householder_by_tract_2017.csv'
download(race_file_name,project)
race_17=pd.read_csv(race_file_name,usecols=[0,3,5,7,9,11,13,15])
race_17.loc[:,'Tract']=race_17.Tract.astype(str)
race_17.set_index('Tract',inplace=True)
print (f"Valid Rows Before Cleansing: {valid_pct(valid_list,'race_17')}%")
race_17.dropna(inplace=True)
race_17.drop_duplicates(inplace=True)
race_17 = clean(race_17)
print('\nrace_17:\n',race_17.head())
race_17.shape

Attempting to get file race_of_householder_by_tract_2017.csv
Downloading...
Completed writing out file
Valid Rows Before Cleansing: 99%

race_17:
         Pct_White  Pct_Black  Pct_Native  Pct_Asian  Pct_Pac_Isl  Pct_Other  \
Tract                                                                         
400100  78.149920   2.392344    0.000000  14.114833     0.000000   0.717703   
400200  85.990338   0.966184    0.603865   8.937198     0.000000   0.000000   
400300  73.270952  11.432059    0.000000  10.821806     0.325468   1.342555   
400400  70.156775   5.879059    0.783875  15.733483     0.000000   2.351624   
400500  60.676923  24.861538    0.000000   4.861538     0.000000   2.523077   

        Pct_Mixed  
Tract              
400100   4.625199  
400200   3.502415  
400300   2.807160  
400400   5.095185  
400500   7.076923  


(7968, 7)

### Prepare ACS 2013-2017 employment status data

In [10]:
emp_file_name = 'employment_status_2017.csv'
download(emp_file_name,project)
emp_17=pd.read_csv(emp_file_name)
emp_17.loc[:,'Tract']=emp_17.Tract.astype(int).astype(str)
emp_17.set_index('Tract',inplace=True)
print (f"Valid Rows Before Cleansing: {valid_pct(valid_list,'emp_17')}%")
emp_17 = clean (emp_17)
print('\nemp_17:\n',emp_17.head())
emp_17.shape

Attempting to get file employment_status_2017.csv
Downloading...
Completed writing out file
Valid Rows Before Cleansing: 99%

emp_17:
         Pct_LF_Part  Pct_EP_Ratio  Pct_Unemp
Tract                                       
101110         62.7          56.6        9.6
101122         75.0          69.3        7.6
101210         69.3          62.4        9.9
101220         57.3          53.2        7.0
101300         54.4          48.8       10.3


(2346, 3)

### Prepare ACS 2013-2017 disability data

In [11]:
dis_file_name = 'disability_2017.csv'
download(dis_file_name,project)
dis_17=pd.read_csv(dis_file_name,dtype=str)
dis_17.set_index('Tract',inplace=True)
print (f"Valid Rows Before Cleansing: {valid_pct(valid_list,'dis_17')}%")
dis_17 = clean(dis_17)
print('\ndis_17:\n',dis_17.head())
dis_17.shape

Attempting to get file disability_2017.csv
Downloading...
Completed writing out file
Valid Rows Before Cleansing: 98%

dis_17:
         Pct_Dis_0-18  Pct_Dis_19-64  Pct_Dis_65+
Tract                                           
101110      3.364486       6.558533    45.034247
101122      8.959538       8.068903    33.928571
101210      3.129161      16.438021    50.762527
101220      0.000000      11.739745    62.500000
101300      8.395522       8.966245    44.405594


(2348, 3)

### Prepare ACS 2013-2017 commute data

In [12]:
com_file_name = 'commute_2017.csv'
download(com_file_name,project)
com_17=pd.read_csv(com_file_name,dtype=str)
com_17.set_index('Tract',inplace=True)
print (f"Valid Rows Before Cleansing: {valid_pct(valid_list,'com_17')}%")
com_17 = clean(com_17)
print('\ncom_17:\n',com_17.head())
com_17.shape

Attempting to get file commute_2017.csv
Downloading...
Completed writing out file
Valid Rows Before Cleansing: 99%

com_17:
         Pct_Hour_Commute  Pct_Male_Hour_Commute  Pct_Female_Hour_Commute  \
Tract                                                                      
101110              20.0                   21.2                     18.6   
101122              18.7                   18.5                     18.8   
101210              11.7                    5.6                     19.2   
101220               6.6                    4.3                      9.5   
101300              15.5                    8.9                     20.9   

        Avg_Min_Commute  Avg_Min_Male_Commute  Avg_Min_Female_Commute  
Tract                                                                  
101110             36.4                  37.0                    35.6  
101122             40.4                  40.2                    40.8  
101210             29.0                  27.2         

(2346, 6)

### Prepare ACS 2013-2017 children's health insurance coverage data

In [13]:
ins_file_name = 'health_insurance_2017.csv'
download(ins_file_name,project)
ins_17=pd.read_csv(ins_file_name,dtype=str)
ins_17.set_index('Tract',inplace=True)
print (f"Valid Rows Before Cleansing: {valid_pct(valid_list,'ins_17')}%")
ins_17 = clean(ins_17)
print('\nins_17:\n',ins_17.head())
ins_17.shape

Attempting to get file health_insurance_2017.csv
Downloading...
Completed writing out file
Valid Rows Before Cleansing: 100%

ins_17:
         Pct_Male_Ins  Pct_Female_Ins
Tract                               
400100         100.0      100.000000
400200         100.0      100.000000
400300         100.0       98.595506
400400         100.0       81.987578
400500         100.0       98.130841


(8059, 2)

### Prepare ACS 2013-2017 marital status data

In [14]:
mar_file_name = 'marital_status_2017.csv'
download(mar_file_name,project)
mar_17=pd.read_csv(mar_file_name,dtype=str)
mar_17.set_index('Tract',inplace=True)
print (f"Valid Rows Before Cleansing: {valid_pct(valid_list,'mar_17')}%")
mar_17 = clean(mar_17)
print('\nmar_17:\n',mar_17.head())
mar_17.shape

Attempting to get file marital_status_2017.csv
Downloading...
Completed writing out file
Valid Rows Before Cleansing: 99%

mar_17:
         Pct_Married  Pct_Widowed  Pct_Divorced  Pct_Separated  \
Tract                                                           
101110         46.9          6.0          11.5            1.2   
101122         57.9          4.0           9.5            2.9   
101210         43.7          4.4           8.2            4.4   
101220         48.0          7.4           8.8            2.7   
101300         51.5          6.8          10.3            0.1   

        Pct_Never_Married  
Tract                      
101110               34.5  
101122               25.7  
101210               39.2  
101220               33.2  
101300               31.2  


(2346, 5)

### Prepare ACS 2013-2017 citizenship and nativity (birthplace) data

In [15]:
cit_file_name = 'citizenship_nativity_2017.csv'
download(cit_file_name,project)
cit_17 = pd.read_csv(cit_file_name,dtype=str)
cit_17.set_index('Tract',inplace=True)
print (f"Valid Rows Before Cleansing: {valid_pct(valid_list,'cit_17')}%")
cit_17 = clean(cit_17)
print('\ncit_17:\n',cit_17.head())
cit_17.shape

Attempting to get file citizenship_nativity_2017.csv
Downloading...
Completed writing out file
Valid Rows Before Cleansing: 100%

cit_17:
         Pct_US_Born_Citizen  Pct_US_Terr_Born_Citizen  \
Tract                                                   
101110            65.418309                  0.000000   
101122            67.232376                  0.000000   
101210            50.405428                  0.000000   
101220            48.832335                  0.000000   
101300            61.143524                  0.210035   

        Pct_Foreign_Born_Citizen  Pct_Naturalized_Citizen  Pct_Non_Citizen  
Tract                                                                       
101110                  1.905388                21.309680        11.366623  
101122                  5.450392                24.477807         2.839426  
101210                  1.390038                27.271223        20.933311  
101220                  1.017964                31.766467        18.383234  

(2326, 5)

### Prepare ACS 2013-2017 geographical mobility data

In [16]:
mob_file_name = 'mobility_2017.csv'
download(mob_file_name,project)
mob_17=pd.read_csv(mob_file_name,dtype=str)
mob_17.set_index('Tract',inplace=True)
print (f"Valid Rows Before Cleansing: {valid_pct(valid_list,'mob_17')}%")
mob_17 = clean(mob_17)
print('\nmob_17:\n',mob_17.head())
mob_17.shape

Attempting to get file mobility_2017.csv
Downloading...
Completed writing out file
Valid Rows Before Cleansing: 99%

mob_17:
         Pct_Moved_In_County  Pct_Moved_Diff_County  Pct_Moved_Diff_State  \
Tract                                                                      
101110                  2.6                    3.3                   0.7   
101122                  4.5                    5.0                   0.0   
101210                  3.7                    0.0                   0.0   
101220                  5.4                    0.3                   0.5   
101300                  4.7                    0.0                   0.8   

        Pct_Moved_Intl  
Tract                   
101110             0.0  
101122             0.0  
101210             0.0  
101220             1.6  
101300             0.0  


(2346, 4)

### Join all dataframes by census tract number

In [17]:
# Combine all dataframes
abs_18_joined=abs_18.copy()
abs_18_joined=abs_18_joined.join(inc_17,how='left')
abs_18_joined=abs_18_joined.join(edu_17,how='left')
abs_18_joined=abs_18_joined.join(eng_17,how='left')
abs_18_joined=abs_18_joined.join(race_17,how='left')
abs_18_joined=abs_18_joined.join(emp_17,how='left')
abs_18_joined=abs_18_joined.join(dis_17,how='left')
abs_18_joined=abs_18_joined.join(com_17,how='left')
abs_18_joined=abs_18_joined.join(ins_17,how='left')
abs_18_joined=abs_18_joined.join(mar_17,how='left')
abs_18_joined=abs_18_joined.join(cit_17,how='left')
abs_18_joined=abs_18_joined.join(mob_17,how='left')

# Eliminate duplicates
abs_18_joined=abs_18_joined[(np.invert(abs_18_joined.index.duplicated(keep='last')))]

# Fill nulls 
#abs_18_joined.fillna(abs_18_joined.mean(),inplace=True)
abs_18_joined.interpolate(method='linear',inplace=True)

print('****Final Joined Dataset***')
print('Total Rows:\t\t',abs_18_joined.shape[0])
print (f"Valid Rows After Joins:\t {valid_pct(valid_list,'abs_18_joined')}%")
abs_18_joined.head()

****Final Joined Dataset***
Total Rows:		 2158
Valid Rows After Joins:	 100%


,Year,Percent,Count,Total,Tract_Nbr,Income,Pct_HS,Pct_Bach,Pct_Eng,Pct_White,...,Pct_Never_Married,Pct_US_Born_Citizen,Pct_US_Terr_Born_Citizen,Pct_Foreign_Born_Citizen,Pct_Naturalized_Citizen,Pct_Non_Citizen,Pct_Moved_In_County,Pct_Moved_Diff_County,Pct_Moved_Diff_State,Pct_Moved_Intl
Tract,,,,,,,,,,,,,,,,,,,,,
101110,2018,11.076923,36,325,101110,51209.0,78.6,21.5,78.3,77.489177,...,34.5,65.418309,0.000000,1.905388,21.309680,11.366623,2.6,3.3,0.7,0.0
101122,2018,9.251102,21,227,101122,85460.0,91.8,25.7,88.7,86.359901,...,25.7,67.232376,0.000000,5.450392,24.477807,2.839426,4.5,5.0,0.0,0.0
101210,2018,11.466666,43,375,101210,34627.0,74.1,16.6,64.0,80.000000,...,39.2,50.405428,0.000000,1.390038,27.271223,20.933311,3.7,0.0,0.0,0.0
101220,2018,7.203390,17,236,101220,40273.0,79.4,18.7,68.2,66.348449,...,33.2,48.832335,0.000000,1.017964,31.766467,18.383234,5.4,0.3,0.5,1.6
101300,2018,7.453416,36,483,101300,81076.0,86.1,30.3,85.6,89.011748,...,31.2,61.143524,0.210035,1.470245,32.648775,4.527421,4.7,0.0,0.8,0.0


### Save joined dataframe to object storage as project asset

In [21]:
joined_file_name = r'la_county_2018_chronic_absence_rates_with_predictor_variables.csv'
joined_csv_data = abs_18_joined.to_csv()
project.save_data(joined_file_name, joined_csv_data, set_project_asset=True, overwrite=True)

{'file_name': 'la_county_2018_chronic_absence_rates_with_predictor_variables.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'iverpyspark-donotdelete-pr-ysp8udweullapt',
 'asset_id': '202734ea-4107-4400-a948-b100c7ae3719'}

### Save data quality assessment results to object storage as project asset

In [22]:
valid_df = pd.DataFrame(valid_list)
valid_df
valid_pct_file_name = 'valid_pct.csv'
valid_pct_csv_data = valid_df.to_csv(index=False)
project.save_data(valid_pct_file_name, valid_pct_csv_data, set_project_asset=True, overwrite=True)

{'file_name': 'valid_pct.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'iverpyspark-donotdelete-pr-ysp8udweullapt',
 'asset_id': '5ca27496-e907-4512-abdf-ad18e8900706'}